In [17]:
%pip install tensorflow

  Using cached wheel-0.43.0-py3-none-any.whl.metadata (2.2 kB)
     ---------------------------------------- 0.0/46.2 kB ? eta -:--:--
     ---------------------------------------- 46.2/46.2 kB 1.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/377.0 MB ? eta -:--:--
   ---------------------------------------- 0.2/377.0 MB 5.8 MB/s eta 0:01:06
   ---------------------------------------- 0.4/377.0 MB 4.8 MB/s eta 0:01:19
   ---------------------------------------- 0.7/377.0 MB 5.2 MB/s eta 0:01:12
   ---------------------------------------- 1.0/377.0 MB 5.9 MB/s eta 0:01:04
   ---------------------------------------- 1.4/377.0 MB 6.1 MB/s eta 0:01:02
   ---------------------------------------- 1.7/377.0 MB 6.3 MB/s eta 0:01:00
   ---------------------------------------- 2.1/377.0 MB 6.5 MB/s eta 0:00:58
   ---------------------------------------- 2.4/377.0 MB 6.6 MB/s eta 0:00:57
   ---------------------------------------- 2.8/377.0 MB 6.8 MB/s eta 0:00:56
   ---------

In [27]:
import tensorflow as tf

# Define model configuration
input_size = 3
number_of_neurons = 3
learning_rate = 0.001
batch_size = 2  # Define the batch size

# Input data
inputs = tf.constant([[1.0, 3.0, 1.0], [1.0, 1.0, 2.0]], dtype=tf.float32)
targets = tf.constant([[2.0, 0.3, -1.9], [1.3, -0.7, 0.1]], dtype=tf.float32)

# Create a TensorFlow dataset from the inputs and targets
dataset = tf.data.Dataset.from_tensor_slices((inputs, targets))
dataset = dataset.batch(batch_size)  # Batch the data

# Define the model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(number_of_neurons,
                          input_shape=(input_size,),
                          activation='linear',
                          use_bias=True)
])

# Initialize weights and bias
initial_weights = tf.constant([
              [0.1, 0.3, 0.2],
              [0.4, 0.2, -0.7],
              [0.1, -0.8, 0.5]
            ], dtype=tf.float32)
initial_bias = tf.ones([number_of_neurons], dtype=tf.float32)
model.layers[0].set_weights([initial_weights, initial_bias])

# Define optimizer and loss function
optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
loss_function = tf.keras.losses.MeanSquaredError()

# Function to perform training step
def train_step(model, inputs, targets):
    with tf.GradientTape() as tape:
        predictions = model(inputs, training=True)
        loss = loss_function(targets, predictions)
    # Only update the weights (first variable in trainable_variables)
    weights = model.trainable_variables[0]  # This assumes the first variable is the weight matrix
    gradients = tape.gradient(loss, [weights])
    optimizer.apply_gradients(zip(gradients, [weights]))
    return loss

# Training loop
for inputs_batch, targets_batch in dataset:
    loss = train_step(model, inputs_batch, targets_batch)
    print("Batch loss:", loss.numpy())

# Final check of weights and bias
updated_weights = model.layers[0].get_weights()[0]
updated_bias = model.layers[0].get_weights()[1]
print("Updated weights:\n", updated_weights)
print("Updated bias (should remain unchanged):\n", updated_bias)


Batch loss: 0.92166674
Updated weights:
 [[ 0.09973334  0.29953334  0.19903333]
 [ 0.39946666  0.199      -0.70196664]
 [ 0.0996     -0.8006667   0.49856666]]
Updated bias (should remain unchanged):
 [1. 1. 1.]
